### Introduction

This codelab is based on https://www.rainbowbreeze.it/serving-gemma-2b-for-free-using-google-colab/ blogpost.

Author: Alfredo Morresi

## Install Ollama inside this Colab

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

### Download packages to create the Cloudflare tunnel

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

### Prepare the thread to expose the Ollama via cloudflare

In [ ]:
import os
# Set OLLAMA_HOST to specify bind address
# https://github.com/ollama/ollama/blob/main/docs/faq.md#setting-environment-variables-on-linux
os.environ.update({'OLLAMA_HOST': '0.0.0.0'})

import subprocess
import threading
import time
import socket

def iframe_thread(port):
    print("Entro nel thread")
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    print("Esco dal thread")
    p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print("\n\n\n\n\n")
            print("running ollama server\n\n", l[l.find("http"):], end='')
            print("\n\n\n\n\n")

threading.Thread(target=iframe_thread, daemon=True, args=(11434,)).start()

### Start serving Ollama

In [ ]:
# Once the serve command is launched, the Colab waits indefinitely here for new command
!ollama serve

### Use Ollama API from the command line

Finally, it's time to call the exposed Ollama API, with ```<your_cloudflare_url>``` as something like https://auditor-protect-vids-slots.trycloudflare.com.


Install the gemma:2b model
```
curl <your_cloudflare_url>/api/pull -d '{ "name": "gemma:2b" }'
```

Cache the model, to save some time between calls (by default, after 5 mins the model is discarded from memory)
```
curl <your_cloudflare_url>/api/generate -d '{"model": "gemma:2b", "keep_alive": -1}'
```

Finally, ask for the prompt
```
curl <your_cloudflare_url>/api/generate --max-time 600 -d '{"model": "gemma:2b", "stream":false, "prompt": "generate a bash script to ask the user for a url, for a message, and then call that url using the message as json payload"}' | jq ".response"
```